In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
IMG_WIDTH, IMG_HEIGHT = 224, 224
NUM_CLASSES = 16  
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
NUM_EPOCHS = 10

# pre-trained on ImageNet
base_model = ResNet50(
    weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
)

# Add custom layers
x = base_model.output
x = Conv2D(64, (3, 3), activation='relu')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# ResNet50 model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = True

# Compile model
optimizer = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Load and preprocess data
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/flowers/flowers/',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/flowers/flowers/',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/flowers/flowers/',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=NUM_EPOCHS
)

# Evaluate the model 
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

94765736/94765736 [==============================] - 0s 0us/step
Found 12599 images belonging to 16 classes.
Found 3141 images belonging to 16 classes.
Found 15740 images belonging to 16 classes.
Epoch 1/10
393/393 [==============================] - 143s 242ms/step - loss: 1.4708 - accuracy: 0.8562 - val_loss: 3.9469 - val_accuracy: 0.0686
Epoch 2/10
393/393 [==============================] - 82s 209ms/step - loss: 0.9489 - accuracy: 0.9568 - val_loss: 1.5024 - val_accuracy: 0.7784
Epoch 3/10
393/393 [==============================] - 80s 203ms/step - loss: 0.7608 - accuracy: 0.9736 - val_loss: 0.8826 - val_accuracy: 0.9314
Epoch 4/10
393/393 [==============================] - 83s 210ms/step - loss: 0.6492 - accuracy: 0.9784 - val_loss: 0.8645 - val_accuracy: 0.9193
Epoch 5/10
393/393 [==============================] - 79s 202ms/step - loss: 0.5543 - accuracy: 0.9818 - val_loss: 0.8218 - val_accuracy: 0.9145
Epoch 6/10
393/393 [==============================] - 83s 211ms/step - loss: 0